#Modelo de Machine Learning de Ponta a Ponta

Depois que apresentamos as soluções para melhorar o desempenho da empresa e recuperar uma parte do faturamento, estamos prontos para criar um Modelo de Machine Learning capaz de prever a demanda para os próximos dias, gerando valor para os parceiros e levando informações preciosas para que eles possam se preparar para o dia de trabalho.

In [77]:
#import das bibliotecas
import pandas as pd
import numpy as np

## Análise de Estrutura

## Remoção de Outliers

In [78]:
#Leitura do csv
df_orders = pd.read_csv("orders.csv")

In [79]:
#Corte de outliers
df_orders = df_orders[(df_orders['order_amount'] >= 15) &
          (df_orders['order_amount'] <= 200)]

In [80]:
#info
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333629 entries, 0 to 368998
Data columns (total 29 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   order_id                           333629 non-null  int64  
 1   store_id                           333629 non-null  int64  
 2   channel_id                         333629 non-null  int64  
 3   payment_order_id                   333629 non-null  int64  
 4   delivery_order_id                  333629 non-null  int64  
 5   order_status                       333629 non-null  object 
 6   order_amount                       333629 non-null  float64
 7   order_delivery_fee                 333629 non-null  float64
 8   order_delivery_cost                327275 non-null  float64
 9   order_created_hour                 333629 non-null  int64  
 10  order_created_minute               333629 non-null  int64  
 11  order_created_day                  333629 no

## Novas Features

In [81]:
#converter order_moment_created para data
df_orders['order_moment_created'] = pd.to_datetime(df_orders['order_moment_created'])

c:\AppData\Local\Temp\ipykernel_12092\2250362991.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_orders['order_moment_created'] = pd.to_datetime(df_orders['order_moment_created'])


In [82]:
#nova coluna com o dia da semana
df_orders['day_of_week'] = df_orders['order_moment_created'].dt.day_of_week
df_orders['day_of_week'].head()

0    4
1    4
2    4
3    4
4    4
Name: day_of_week, dtype: int32

In [83]:
df_orders['order_moment_created'].dt.day_name().head()

0    Friday
1    Friday
2    Friday
3    Friday
4    Friday
Name: order_moment_created, dtype: object

In [84]:
#Análise da quantidade de horas que temos pedidos por dia
df_orders['order_created_hour'].value_counts().reset_index().sort_values('order_created_hour')

,order_created_hour,count
6,0,25126
12,1,4002
13,2,1003
15,3,713
18,4,267
17,5,417
19,6,260
21,7,214
23,8,124
22,9,126


In [85]:
#criando função por faixa de horário
def faixa_horario(hora):
    if hora>= 0 and hora <= 5:
        return 'madrugada'
    elif hora >= 6 and hora <= 10:
        return 'manha'
    elif hora >= 11 and hora <= 14:
        return 'almoco'
    elif hora >= 15 and hora <= 18:
        return 'tarde'
    else:
        return 'noite'

faixa_horario(19)

'noite'

In [86]:
#aplicando função e criando uma nova coluna com o nome de faixa de horário
df_orders['faixa_horario'] = df_orders['order_created_hour'].apply(lambda x: faixa_horario(x))


In [87]:
#distribuição da faixa de horário
df_orders['faixa_horario'].value_counts()

faixa_horario
noite        144269
tarde        119598
almoco        37294
madrugada     31528
manha           940
Name: count, dtype: int64

## DataPrep

In [88]:
#Criando um novo dataset de orders_treatment
df_orders_treatment = df_orders.copy()

In [89]:
#info
df_orders_treatment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333629 entries, 0 to 368998
Data columns (total 31 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   order_id                           333629 non-null  int64         
 1   store_id                           333629 non-null  int64         
 2   channel_id                         333629 non-null  int64         
 3   payment_order_id                   333629 non-null  int64         
 4   delivery_order_id                  333629 non-null  int64         
 5   order_status                       333629 non-null  object        
 6   order_amount                       333629 non-null  float64       
 7   order_delivery_fee                 333629 non-null  float64       
 8   order_delivery_cost                327275 non-null  float64       
 9   order_created_hour                 333629 non-null  int64         
 10  order_created_minute     

### Data Cleaning

#### Exclusão de colunas

In [90]:
#após realizar análise das variáveis excluir variável order_moment_delivered
df_orders_treatment.drop('order_moment_delivered', axis=1, inplace=True)

In [91]:
#definição das colunas que serão excluídas da nossa base
columns_delete = ['payment_order_id',
'delivery_order_id',
'order_status',
'order_created_minute',
'order_created_month',
'order_created_year',
'order_moment_created',
'order_moment_accepted',
'order_moment_ready',
'order_moment_collected',
'order_moment_in_expedition',
'order_moment_delivering',
'order_moment_finished',
'order_delivery_fee',
'order_delivery_cost',
'order_created_hour']

In [92]:
#excluir colunas
df_orders_treatment.drop(columns_delete, axis=1, inplace=True)

#### Prenchendo valores nulos

In [93]:
#info
df_orders_treatment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333629 entries, 0 to 368998
Data columns (total 14 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   order_id                           333629 non-null  int64  
 1   store_id                           333629 non-null  int64  
 2   channel_id                         333629 non-null  int64  
 3   order_amount                       333629 non-null  float64
 4   order_created_day                  333629 non-null  int64  
 5   order_metric_collected_time        290223 non-null  float64
 6   order_metric_paused_time           272423 non-null  float64
 7   order_metric_production_time       312151 non-null  float64
 8   order_metric_walking_time          269553 non-null  float64
 9   order_metric_expediton_speed_time  304793 non-null  float64
 10  order_metric_transit_time          312372 non-null  float64
 11  order_metric_cycle_time            320707 no

In [94]:
#utilização do fillna com a mediana para preencher os valores do order_metric_collected_time
df_orders_treatment['order_metric_collected_time'].fillna(df_orders_treatment['order_metric_collected_time'].median(), inplace=True)

c:\AppData\Local\Temp\ipykernel_12092\1905203620.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_orders_treatment['order_metric_collected_time'].fillna(df_orders_treatment['order_metric_collected_time'].median(), inplace=True)


### Criação do dataset final

In [95]:
#Criação do dataset final de grupos
df_orders_treatment_group = df_orders_treatment.groupby(['store_id', 'channel_id', 'order_created_day', 'day_of_week', 'faixa_horario']) \
  .agg({'order_metric_collected_time':'median',
        'order_metric_paused_time':'median',
        'order_metric_production_time':'median',
        'order_metric_walking_time':'median',
        'order_metric_expediton_speed_time':'median',
        'order_metric_transit_time':'median',
        'order_metric_cycle_time':'median',
        'order_id':'count'}) \
  .reset_index() \
  .sort_values('order_id', ascending=False)

df_orders_treatment_group=df_orders_treatment_group.rename(columns = {'order_id':'demanda'})

df_orders_treatment_group.head()

,store_id,channel_id,order_created_day,day_of_week,faixa_horario,order_metric_collected_time,order_metric_paused_time,order_metric_production_time,order_metric_walking_time,order_metric_expediton_speed_time,order_metric_transit_time,order_metric_cycle_time,demanda
22931,676,5,26,4,noite,1.600,2.930,30.135,2.300,5.950,15.320,52.150,182
22766,676,5,12,4,noite,1.600,3.475,34.250,1.840,6.250,14.185,57.880,157
22707,676,5,7,6,tarde,2.500,2.330,30.035,2.880,6.375,13.450,49.785,154
22956,676,5,28,6,tarde,1.600,2.875,33.520,1.640,5.170,12.470,52.760,147
24297,707,5,26,4,noite,1.345,1.570,21.525,4.465,6.075,16.025,48.625,146


In [166]:
#info
df_orders_treatment_group.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66624 entries, 22931 to 66623
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   store_id                           66624 non-null  int64  
 1   channel_id                         66624 non-null  int64  
 2   order_created_day                  66624 non-null  int64  
 3   day_of_week                        66624 non-null  int32  
 4   faixa_horario                      66624 non-null  object 
 5   order_metric_collected_time        66624 non-null  float64
 6   order_metric_paused_time           57376 non-null  float64
 7   order_metric_production_time       62378 non-null  float64
 8   order_metric_walking_time          57516 non-null  float64
 9   order_metric_expediton_speed_time  61213 non-null  float64
 10  order_metric_transit_time          62402 non-null  float64
 11  order_metric_cycle_time            63752 non-null  floa

### Criando um conjunto de testes

Antes de seguir adiante, vamos precisar criar um conjunto de teste, colocá-lo de lado e nunca checá-lo. <br/>

Quando estimamos o erro de generalização utilizando o conjunto de teste, sua estimativa será muito otimista e será lançado um sistema que não funcionará tão bem quanto o esperado.  <br/>

Isso é chamado de **data snooping bias.** <br/>

O Scikit-Learn fornece algumas funções para dividir conjuntos de dados em vários subconjuntos de diversas maneiras. A função mais simples é train_test_split.

In [167]:
#import do train_test_split
from sklearn.model_selection import train_test_split as tss

In [183]:
#Separa a base de treino e teste
train_set, test_set = tss(df_orders_treatment_group, test_size=0.2, random_state=42)
df_orders_train = train_set.copy()
df_orders_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53299 entries, 49037 to 9950
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   store_id                           53299 non-null  int64  
 1   channel_id                         53299 non-null  int64  
 2   order_created_day                  53299 non-null  int64  
 3   day_of_week                        53299 non-null  int32  
 4   faixa_horario                      53299 non-null  object 
 5   order_metric_collected_time        53299 non-null  float64
 6   order_metric_paused_time           45895 non-null  float64
 7   order_metric_production_time       49878 non-null  float64
 8   order_metric_walking_time          45998 non-null  float64
 9   order_metric_expediton_speed_time  48955 non-null  float64
 10  order_metric_transit_time          49934 non-null  float64
 11  order_metric_cycle_time            51003 non-null  float

In [184]:
#Separa a Label principal
df_orders_tratment_label = df_orders_train[['demanda']].copy()

In [185]:
#Remove a Label da base
df_orders_treatment = df_orders_train.drop('demanda', axis=1)

In [186]:
#Seleciona as variáveis float64
df_orders_float64 = df_orders_treatment.select_dtypes(np.float64).copy()
df_orders_float64.head()

,order_metric_collected_time,order_metric_paused_time,order_metric_production_time,order_metric_walking_time,order_metric_expediton_speed_time,order_metric_transit_time,order_metric_cycle_time
49037,1.47,2.150,20.47,2.450,5.05,15.870,36.45
23762,3.20,3.035,18.80,4.155,7.60,18.695,48.59
43636,5.02,53.200,5.03,6.630,59.83,58.370,123.23
6376,2.30,0.970,20.37,6.820,7.80,10.020,41.18
288,0.67,2.270,14.62,0.800,3.05,12.820,32.67


In [188]:
#Import do SimpleImputer
from sklearn.impute import SimpleImputer

In [190]:
#Adicionando estratégia da mediana no imputer
imputer = SimpleImputer(strategy='median')

In [191]:
#Treina o imputer
imputer.fit(df_orders_float64)

SimpleImputer(strategy='median')

In [194]:
#Cria nova matriz preenchendo os valores nulos com o imputer
float_vars = imputer.transform(df_orders_float64)
float_vars

array([[  1.47 ,   2.15 ,  20.47 , ...,   5.05 ,  15.87 ,  36.45 ],
       [  3.2  ,   3.035,  18.8  , ...,   7.6  ,  18.695,  48.59 ],
       [  5.02 ,  53.2  ,   5.03 , ...,  59.83 ,  58.37 , 123.23 ],
       ...,
       [  2.72 ,   0.35 ,   1.7  , ...,   5.35 ,  14.58 ,  21.63 ],
       [  1.215,   5.77 ,  15.3  , ...,   8.32 ,  18.45 ,  42.89 ],
       [  0.58 ,   4.05 ,  11.88 , ...,   7.9  ,  17.68 ,  46.57 ]])

In [196]:
#Cria novo dataset para apresentar os valores
df_float64 = pd.DataFrame(float_vars, columns=df_orders_float64.columns, index=df_orders_float64.index)

df_float64.head()

,order_metric_collected_time,order_metric_paused_time,order_metric_production_time,order_metric_walking_time,order_metric_expediton_speed_time,order_metric_transit_time,order_metric_cycle_time
49037,1.47,2.150,20.47,2.450,5.05,15.870,36.45
23762,3.20,3.035,18.80,4.155,7.60,18.695,48.59
43636,5.02,53.200,5.03,6.630,59.83,58.370,123.23
6376,2.30,0.970,20.37,6.820,7.80,10.020,41.18
288,0.67,2.270,14.62,0.800,3.05,12.820,32.67


In [197]:
#info do novo dataset
df_float64.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53299 entries, 49037 to 9950
Data columns (total 7 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   order_metric_collected_time        53299 non-null  float64
 1   order_metric_paused_time           53299 non-null  float64
 2   order_metric_production_time       53299 non-null  float64
 3   order_metric_walking_time          53299 non-null  float64
 4   order_metric_expediton_speed_time  53299 non-null  float64
 5   order_metric_transit_time          53299 non-null  float64
 6   order_metric_cycle_time            53299 non-null  float64
dtypes: float64(7)
memory usage: 3.3 MB


## Seleção Final

## Variáveis Categóricas

In [108]:
#info


In [109]:
#seleção de day_of_week e faixa_horario


In [110]:
#value_counts faixa de horario


In [111]:
#novo dataset com a seleção das variaveis numéricas categóricas


In [112]:
#novo dataset com a seleção das variáveis categóricas


In [113]:
#import do OrdinalEncoder e do OneHotEncoder


### Ordinal Encoder
A maioria dos algoritmos de Aprendizado de Máquina prefere trabalhar com números, então vamos converter as categorias de texto para números. Para tanto, podemos utilizar o método OrdinalEncoder(), que mapeia cada categoria para um número inteiro diferente.

In [114]:
#tratamento com o OrdinalEncoder


In [115]:
#Verificação das categorias do OrdinalEncoder


### OneHotEncoder
O OrdinalEncoder pega a quantidade de atributos e converte em números, porém transformando puramente em números ele cria uma diferença de valores entre os números. As categorias 0 e 1 transformadas tem uma distância semelhante, não podemos dizer o mesmo para as categorias 0 e 2, os algoritmos de ML enxergarão essa escala como uma diferença significa entre os dados. <br/>
A utilização do OneHotEncoder é melhor aproveitada para esses casos. Ela cria novos atributos de acordo com a quantidade de atributos com 0 e 1.


In [116]:
#tratamento da categoria com o OneHotEncoder


In [117]:
#Visuaulização das categorias em array


In [118]:
#Visuaulização das categorias


## Escalonando nossos dados

In [119]:
#visualização do df_float64


In [120]:
#import do MinMaxScaler e do StandardScaler


### MinMaxScaler
O escalonamento min-max (muitas pessoas chamam de normalização) é bastante simples: os valores são deslocados e redimensionados para que acabem variando de 0 a 1. Ele subtrai o valor mínimo e divide pelo máximo menos o mínimo. O Scikit-Learn fornece um transformador chamado MinMaxScaler para isso. Ele possui um hiper parâmetro feature_range que permite alterar o intervalo se não quiser 0-1 por algum motivo.

In [121]:
#Tratamento com MinMaxScaler


In [122]:
#Visualiza transformação


### StandardScaler
A padronização é bem diferente: em primeiro lugar ela subtrai o valor médio (assim os valores padronizados sempre têm média zero) e, em seguida, divide pela variância, de modo que a distribuição resultante tenha variância unitária.
Ao contrário do escalonamento min-max, a padronização não vincula valores a um intervalo específico, o que pode ser um problema para alguns algoritmos.
No entanto, a padronização é muito menos afetada por outliers.
O Scikit-Learn fornece um transformador para padronização chamado StandardScaler.

In [123]:
#Tratamento com o SatandardScaler


In [124]:
#Visualiza transformação


## Pipeline

In [125]:
#import do Pipeline


Existem muitas etapas de transformação de dados que precisam ser executadas na ordem correta. Felizmente, o Scikit-Learn fornece a classe Pipeline para ajudar tais sequências de transformações.

O construtor Pipeline se vale de uma lista de pares de nome/estimador que definem uma sequência de etapas. Todos, exceto o último estimador, devem ser transformadores (ou seja, eles devem ter um método fit_transform()).


### Pipeline Numérico

In [126]:
#Criação do novo pipeline com o Imputer e StandardScaler


In [127]:
#Criação de um novo dataset com o pipline numérico


In [128]:
#Apresentação do Shape da transformação após o pipeline


### Pipeline Categório + Full Pipeline

In [129]:
#Import do column transform


In [130]:
#Seleção das variáveis numéricas, categóricas numéricas e categóricas.


In [131]:
#Criação do Full Pipeline


In [132]:
#Transformação dos dados através do full pipeline


In [133]:
#Apresentação do shspe dos dados transformados


## Trienando o modelo

### Regressão Linear
A Regressão Linear é uma técnica estatística usada para modelar a relação entre uma variável dependente (ou alvo) e uma ou mais variáveis independentes (ou preditoras). O objetivo é ajustar uma linha reta (ou plano, no caso de múltiplas variáveis) que melhor represente essa relação, de forma a prever os valores da variável dependente com base nos valores das variáveis independentes.

**Regressão Linear Simples:**<br/>
Utiliza uma única variável independente para prever o valor da variável dependente. A equação da reta é dada por:

$$
y = \beta_0 + \beta_1 x + \epsilon
$$

Onde:
- $( y  )$ é a variável dependente (o que você quer prever),
- \( $\beta_0 $ ) é o intercepto (onde a linha cruza o eixo y),
- \( $\beta_1 $) é o coeficiente angular (influência de \( x \) sobre \( y \)),
- \( x \) é a variável independente (o que você usa para prever),
- \( $\epsilon \$ ) é o erro ou termo de resíduo.

**Regressão Linear Múltipla:** <br/>
Utiliza duas ou mais variáveis independentes para prever o valor da variável dependente. A equação se expande para:

$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \dots + \beta_n x_n + \epsilon
$$



<img src="https://analisemacro.com.br/wp-content/uploads/2023/08/p3.png">

In [134]:
#import e modelo de regressão linear


In [135]:
#Seleciona os dados e as labels e passa pelo full pipeline


In [136]:
#Verifica as predições


In [137]:
#Verifica os valores reais


### RMSE (Root Mean Squared Error)

O **RMSE (Root Mean Squared Error)** ou **Erro Quadrático Médio da Raiz** é uma métrica usada para avaliar a qualidade de um modelo de regressão, especialmente em Machine Learning. Ele mede a diferença entre os valores preditos por um modelo e os valores reais observados. O RMSE dá uma indicação de quão bem o modelo está prevendo os dados.

**Como é calculado o RMSE?** <br/>

O RMSE é calculado em três etapas:

1. **Calcula-se o erro para cada ponto de dados**: A diferença entre o valor observado (real) e o valor previsto pelo modelo.
   
   $$ \text{Erro} = y_{\text{real}} - y_{\text{previsto}} $$

2. **Eleva-se o erro ao quadrado**: Isso é feito para garantir que os erros negativos não cancelem os positivos, e também para dar maior peso a erros maiores.
$$ \text{Erro Quadrático} = (y_{\text{real}} - y_{\text{previsto}})^2 $$

3. **Calcula-se a média dos erros quadráticos**: Soma-se todos os erros quadráticos e divide-se pelo número total de amostras.

   $$ \text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_{\text{real}}^{(i)} - y_{\text{previsto}}^{(i)})^2 $$

4. **Tira-se a raiz quadrada** da média dos erros quadráticos:

   $$\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_{\text{real}}^{(i)} - y_{\text{previsto}}^{(i)})^2}$$

**Interpretação do RMSE:**
- **RMSE = 0**: Indica que o modelo fez previsões perfeitas, ou seja, não houve erro entre os valores previstos e os reais.
- **Valores maiores de RMSE**: Sugerem que o modelo tem maior erro de previsão. Quanto maior o RMSE, pior o modelo se ajusta aos dados.

**Vantagens do RMSE:**
- **Penaliza erros grandes**: Como o erro é elevado ao quadrado, os erros maiores têm um impacto mais significativo na métrica, o que é útil para identificar modelos que cometem grandes erros.

**Limitação:**
- **Sensível a outliers**: Como o erro é elevado ao quadrado, o RMSE é particularmente sensível a outliers (valores muito distantes dos demais), o que pode distorcer a avaliação do modelo.

**Exemplo:**
Se um modelo de regressão está prevendo o preço de casas, o RMSE pode ser usado para avaliar o quão próxima a previsão de preço está em relação ao preço real das casas. Se o RMSE for baixo, significa que as previsões estão próximas dos valores reais; se for alto, indica que as previsões estão dispersas dos valores reais.
<br/><br/>
**Comparação com outras métricas:**
- O RMSE pode ser comparado com o **MAE (Mean Absolute Error)**, que é outra métrica comum. A diferença principal é que o RMSE penaliza mais fortemente os erros grandes, enquanto o MAE trata todos os erros de forma linear.


In [138]:
#Import da métrica erro quadrado médio


### MAE (Mean Absolute Error)

O **MAE (Mean Absolute Error)** ou **Erro Médio Absoluto** é uma métrica de avaliação usada em modelos de regressão. Ele mede a magnitude média dos erros entre os valores previstos pelo modelo e os valores reais, sem considerar a direção do erro (se o valor previsto foi maior ou menor que o valor real).

## Fórmula do MAE:

A fórmula para calcular o MAE é:

$$
MAE = \frac{1}{n} \sum_{i=1}^{n} |y_{\text{real}}^{(i)} - y_{\text{previsto}}^{(i)}|
$$

**Como o MAE funciona:**
- O MAE simplesmente soma a magnitude de todos os erros (diferença entre previsto e real) e divide pelo número total de pontos de dados.
- Como utiliza o valor absoluto, o MAE não se preocupa se o modelo está subestimando ou superestimando as previsões, apenas com a magnitude média do erro.

**Interpretação do MAE:**
- **MAE = 0**: Significa que o modelo fez previsões perfeitas, ou seja, não há diferença entre os valores reais e os previstos.
- **Valores maiores de MAE**: Indicam que as previsões do modelo estão, em média, mais distantes dos valores reais.

**Vantagens do MAE:**
- **Fácil de interpretar**: O MAE representa o erro médio em unidades da variável prevista, facilitando a compreensão do quão distantes estão as previsões em relação aos valores reais.
- **Robustez**: O MAE é menos sensível a outliers (valores extremos) comparado ao RMSE, já que não eleva os erros ao quadrado.

**Limitações do MAE:**
- **Não penaliza grandes erros com intensidade**: Como não eleva os erros ao quadrado (diferente do RMSE), o MAE trata todos os erros de forma igual. Portanto, grandes erros não são penalizados de forma mais acentuada, o que pode ser uma desvantagem em alguns contextos.

**Exemplo de uso do MAE:**
Se você estiver prevendo o preço de casas, o MAE pode indicar o erro médio em dólares entre as previsões do modelo e os preços reais. Por exemplo, um **MAE de 5.000 dólares** significa que, em média, o modelo erra em 5.000 dólares ao prever o preço de uma casa.

**Comparação com RMSE:**
- **MAE** trata todos os erros de forma linear, sem dar mais peso aos erros maiores.
- **RMSE** penaliza mais fortemente os grandes erros porque eleva os erros ao quadrado antes de calcular a média.

Ambas as métricas são úteis, e a escolha entre uma e outra depende do tipo de análise que você deseja fazer. O **MAE** é mais simples e intuitivo, enquanto o **RMSE** é mais sensível a grandes erros.


In [139]:
#Import da métrica MAE

# Gera as previsões

# Calcula o MAE



In [140]:
#Describe dos Labels


## Árvore de Decisão (DecisionTree)

A Árvore de Decisão é um algoritmo de aprendizado supervisionado usado tanto para problemas de classificação quanto de regressão. Ela funciona como um conjunto de regras de decisão, onde os dados são divididos em subgrupos com base em características específicas. É chamada de "árvore" porque a estrutura se assemelha a um diagrama em forma de árvore, com um nó raiz no topo, nós internos que representam testes em atributos/características, e folhas que representam as decisões finais ou previsões.

Estrutura da Árvore de Decisão:
- **Nó raiz (Root Node):** É o nó inicial que contém o conjunto completo de dados e é o ponto de partida para a construção da árvore. Este nó representa a divisão do primeiro atributo.

- **Nós internos (Internal Nodes):** Cada nó interno representa um teste em um atributo. Aqui, os dados são divididos com base em uma condição de decisão (por exemplo, "Salário > 50K").

- **Galhos (Branches):** São os caminhos que conectam os nós. Cada ramo representa o resultado de um teste e direciona o fluxo de dados para os nós filhos.

- **Folhas (Leaf Nodes):** Os nós folha representam a classe ou valor final de previsão. No caso de classificação, cada folha terá uma classe; no caso de regressão, terá um valor numérico.

In [141]:
#import da árvore de decisão


In [142]:
#Treinamento da Árvore de Decisão


In [143]:
#Realiza as previsões e testa a métrica do RMSE


In [144]:
#Dados Predidos


In [145]:
#dados reais


## Árvores Aleatória (Randon Forest)

As Árvores Aleatórias (ou Random Forest) são uma extensão do algoritmo de Árvores de Decisão que combinam várias árvores de decisão para melhorar a precisão do modelo e reduzir o risco de overfitting (superajuste). Essa técnica é amplamente utilizada tanto para tarefas de classificação quanto de regressão e é conhecida por ser robusta, precisa e fácil de interpretar.

O que é uma Random Forest?
Uma Random Forest é composta por um conjunto de árvores de decisão independentes (daí o nome "floresta"), onde cada árvore é treinada em uma amostra diferente dos dados e com um subconjunto aleatório dos atributos. As previsões finais são feitas com base no consenso das árvores, o que reduz a variância e melhora a generalização do modelo.

- Para classificação, a Random Forest usa a votação majoritária: cada árvore faz uma previsão e a classe mais votada é escolhida como a previsão final.
- Para regressão, a Random Forest faz a média das previsões das várias árvores.

In [146]:
#import do random forest


In [147]:
#treinamento do random forest


In [148]:
#Realiza as previsões e testa a métrica do RMSE


In [149]:
#predições


In [150]:
#valores reais


## Modelo Final

Agora que já definimos o nosso modelo com a melhor métrica, podemos fazer criar o modelo de Machine Learning

In [151]:
#seleciona o dataset de teste


In [152]:
#seleciona o modelo final


In [153]:
# Usando o conjunto de teste para a avaliação


In [154]:
# Transformando os dados de teste com o pipeline usado no treinamento


In [155]:
# Fazendo previsões com o modelo final


In [156]:
# Calculando as métricas de desempenho

# Exibindo os resultados


In [157]:
#describe y_test


In [158]:
#avalia os dados de entrada


In [159]:
#o valor real


In [160]:
#previsão final


## Export do Modelo

No contexto de aprendizado de máquina, exportar um modelo refere-se ao processo de salvar o estado de um modelo treinado em um arquivo, permitindo que ele seja reutilizado posteriormente sem a necessidade de re-treinamento.

In [161]:
#import do pickle


In [162]:
#realiza o dumping do modelo


## Import do Modelo

In [163]:
#import do modelo


In [164]:
#faz a predição do X_test


In [165]:
#traz as métricas do rmse e mae
